In [ ]:
pip install -r ./requirements.txt

In [ ]:
import os
import pandas as pd
import shutil

%cd ..

import api.constants as constants

In [ ]:
%cd ./notebook

shutil.rmtree(constants.CSV_EXTRACTED_PATH)
shutil.rmtree(constants.CSV_MERGED_PATH)

In [ ]:
!bash ../scripts/unzip.sh

In [5]:
os.mkdir(constants.CSV_MERGED_PATH)

In [6]:
for type in constants.TYPES.keys():
    df = pd.DataFrame()

    for file in os.listdir(constants.CSV_EXTRACTED_PATH):
        if file.endswith(f"{type}.csv"):
            file_df = pd.read_csv(
                f"{constants.CSV_EXTRACTED_PATH}/{file}",
                encoding="latin1",
                engine="python",
            )

            file_df = file_df[constants.TYPES[type].values()]

            if df.empty:
                df = file_df
            else:
                df = pd.concat([df, file_df])

    df.to_csv(
        f"{constants.CSV_MERGED_PATH}/{type}.csv",
        encoding="latin1",
        index=False,
        sep=",",
    )